In [ ]:
import gqr
import pandas as pd
from datetime import datetime
import ollama
from tqdm import tqdm
import os

from util import system_prompt, user_prompt

## Load Prompts
Load system and user prompts from prompt.txt

# GQR-Bench: Multi-LLM Comparison
Compare multiple Ollama models on the GQR benchmark dataset

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'


In [ ]:
# List of models to test
models_to_test = [
    # "gemma3:27b",
    # "gemma3:1b",
    # "gemma3:4b",
    # "gemma3:270m",
    # "granite3.3:8b",
    # "granite3.3:2b",
    # "llama3.2:3b",
    # "mistral:7b",
    # "phi3:3.8b",
    # "phi3:14b",
    # "phi4:14b",
    # "gpt-oss:20b",
    # "gpt-oss-safeguard:20b",
    # "phi4-mini:3.8b",
    # "qwen3:14b",
    "qwen3:8b",
    "llama3.3:70b",
]

## Model Configuration
Define all Ollama models to test

In [ ]:
# Global progress bar for tracking prompts
current_pbar = None
prompt_latencies = []

def scoring_function(text: str) -> int:
    global current_pbar, current_model_name, prompt_latencies
    
    try:
        # Format the user prompt with the text
        formatted_user_prompt = user_prompt.format(query=text)
        
        # Track latency for this prompt
        prompt_start = datetime.now()
        
        # Call Ollama with system and user messages
        response = ollama.chat(
            model=current_model_name,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': formatted_user_prompt}
            ],
            think='low' if current_model_name.startswith('gpt-oss') else False,
            stream=False
        )
        
        prompt_end = datetime.now()
        latency = (prompt_end - prompt_start).total_seconds()
        prompt_latencies.append(latency)
        
        # Get the response content
        result = response['message']['content'].strip().lower()
        
        # Map category names to integers
        category_map = {
            'law': 0,
            'finance': 1,
            'health': 2,
            'other': 3
        }
        
        # Return the corresponding integer
        predicted_label = 3  # Default to 'other'
        for category, value in category_map.items():
            if category in result:
                predicted_label = value
                break
        
        # Update progress bar if provided
        if current_pbar:
            current_pbar.update(1)
        
        return predicted_label
        
    except Exception as e:
        print(f"Error with model {current_model_name}: {e}")
        if current_pbar:
            current_pbar.update(1)
        return 3

## Run Benchmarks
Loop through all models and evaluate each one

In [ ]:
result_columns = ["model", "avg_latency", "id_acc", "ood_acc", "gqr_score", "dataset_acc"]

results = []
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_dir = '../results'
os.makedirs(results_dir, exist_ok=True)
output_file = f'{results_dir}/benchmark_results_{timestamp}.csv'

for model_name in models_to_test:
    print()
    print("=" * 60)
    print(f"Testing model: {model_name}")
    print("=" * 60)

    try:
        current_model_name = model_name
        prompt_latencies = []

        id_test_data = gqr.load_id_test_dataset()
        ood_test_data = gqr.load_ood_test_dataset()
        total_size = len(id_test_data) + len(ood_test_data)

        start_time = datetime.now()
        with tqdm(total=total_size, desc=f"{model_name}") as pbar:
            current_pbar = pbar
            id_test_data["predictions"] = [scoring_function(doc) for doc in id_test_data["text"].values]
            ood_test_data["predictions"] = [scoring_function(doc) for doc in ood_test_data["text"].values]
        elapsed_time = (datetime.now() - start_time).total_seconds()

        id_scores = gqr.evaluate(id_test_data["predictions"], ground_truth=id_test_data["label"])
        id_acc = id_scores["accuracy"]

        ood_scores_df = gqr.evaluate_by_dataset(
            ood_test_data, pred_col="predictions", true_col="label", dataset_col="dataset"
        )
        ood_acc = ood_scores_df["accuracy"].mean()

        gqr_score = 2 * (id_acc * ood_acc) / (id_acc + ood_acc) if (id_acc + ood_acc) > 0 else 0.0
        avg_latency = sum(prompt_latencies) / len(prompt_latencies) if prompt_latencies else None
        latency_display = f"{avg_latency:.3f}s" if avg_latency is not None else "n/a"

        dataset_acc = dict(zip(ood_scores_df["dataset"], ood_scores_df["accuracy"]))

        result = {
            "model": model_name,
            "avg_latency": avg_latency,
            "id_acc": id_acc,
            "ood_acc": ood_acc,
            "gqr_score": gqr_score,
            "dataset_acc": str(dataset_acc),
        }
        results.append(result)

        print()
        print(
            f"ID: {id_acc:.4f} | OOD: {ood_acc:.4f} | GQR: {gqr_score:.4f} | Latency: {latency_display}"
        )
        print(f"Per-dataset: {dataset_acc}")

        pd.DataFrame(results, columns=result_columns).to_csv(output_file, index=False)
        print(f"Saved to: {output_file}")

    except Exception as e:
        print(f"Failed: {e}")
        results.append({
            "model": model_name,
            "avg_latency": None,
            "id_acc": None,
            "ood_acc": None,
            "gqr_score": None,
            "dataset_acc": None,
        })
        pd.DataFrame(results, columns=result_columns).to_csv(output_file, index=False)

print()
print("=" * 60)
print("All models tested!")
print("=" * 60)
